<a href="https://colab.research.google.com/github/ankittejyadav/AI/blob/main/Exercises/15_Transfer_Learning_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Transformers with HuggingFace

© Data Trainers LLC. GPL v 3.0.

Author: Axel Sirota

HuggingFace is a company with a heavy open source philosophy that makes transformers readily available so you don't have to do what we did before for every application.

## Prep

In [1]:
!pip install -U datasets evaluate transformers transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import multiprocessing
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

import sys
import keras.backend as K
import random
import os
import pandas as pd
import warnings
import time

TRACE = False
PATIENCE = 2
EPOCHS = 3
BATCH_SIZE = 256

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')

## Tokenizing and loading the dataset

In HuggingFace there are many models, and each has its own tokenizer. Lucky for us there is a class `AutoTokenizer` that does the heavylifting after we provide a checkpoint

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("imdb")  # load imdb dataset
raw_datasets

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Notice it is a dict object with the train, test, and unsupervised datasets to play around

In [4]:
raw_datasets['train'][0]  # Let's see the first review

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

How do we know if it's positive or negative from label=0?

In [5]:
raw_datasets['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

There it is, within features we see that the index 0 is **Negative**

Now to tokenise the dataset we need to load the proper tokenizer for the model we care about. And the we are goin to apply it everywhere!

After this step the tokenizer converts the text into a Tensor of ids, each representing a diferent word in the BERT vocabulary

In [7]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)   # Fetch the tokenizer for that checkpoint


def tokenize_function(example):
    # We are using the BERT tokenizer, specifying to PAD until the end,
    # truncate if either 128 elements are met or the maximum from the model, which you get from the model card

    return tokenizer(example['text'], padding="max_length", truncation=True) # Return a tokenizer function that adds padding to 128 chars and truncates from the examples


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Let's see how it worked!

In [8]:
tokenized_datasets['train'][0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [9]:
tokenizer(tokenized_datasets['train'][0]['text'])

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

The tokenizer from BERT (well DistillBERT) converts each word into its ID according to *its* vocabulary. And notice the masking says we haven't been truncated. What we will do know is do this for all data and convert it into a TF Datasets object (which Keras accepts)

In [10]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset() # Convert the tokenized_datasets["train"] to a TF Dataset


tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset() # Same with validation

In [11]:
for inputs, labels in tf_train_dataset.take(1):
  print(f' inputs: {inputs.shape}, labels: {labels.shape}')


ValueError: too many values to unpack (expected 2)

## Downloading the model and prepare for training

Now let's download the model. It is very important you use the class that starts with `TFAutoModel`. There are auto models for most tasks, so you don't have to manually add the header, for example the `TFAutoModelForSequenceClassification` adds a Dense layer (WITHOUT SOFTMAX) to do the classification

In [ ]:

model = None # Download the model for sequence classification with 2 labels (sentiment analysis)

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = BATCH_SIZE
num_epochs = EPOCHS
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=1e-8, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [ ]:
loss = None  # Set the loss
# Compile the model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=PATIENCE)


In [ ]:
model.summary()

Oh no! We have too many parameters to train! Luckily in Keras is very easy to set some layers as not trainable

In [ ]:
# Set the first layer as non trainable

In [ ]:
model.summary()

*Voilá!*

In [ ]:
# Fit the model

Now we have a trained model that did transfer learning from DistillBERT

## Testing it out!

In [ ]:
tokens = tokenizer(["This is the worst internet service provider", "Although most people say this is the worst, I like it"], padding=True, truncation=True, max_length=128)

In [ ]:
tokens

In [ ]:
model.predict(tokens['input_ids'])

Notice the prediction where not probabilities but logits!

In [ ]:
tf.math.softmax(model.predict(tokens['input_ids'])['logits'])

In [ ]:
tf.math.argmax(tf.math.softmax(model.predict(tokens['input_ids'])['logits']))

And the model was correct!!

In [ ]:
model.evaluate(tf_validation_dataset)